In [ ]:
import glob
import cv2
import mediapipe as mp
from tqdm.notebook import tqdm

img_paths = glob.glob('path2images/*')
path2save = ''

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

In [ ]:
eye_crop = image = None
x_min = x_max = y_min = y_max = 0
x_min_eye = x_max_eye = y_min_eye = y_max_eye = 0
x_coords = y_coords = None
def extract_eye_region(path, save_path):
    global eye_crop, x_min, x_max, y_min, y_max, image, x_coords, y_coords, x_min_eye, x_max_eye, y_min_eye, y_max_eye
    # Load image
    image = cv2.imread(path)
    try:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except :
        # print("Error converting image to RGB. Skipping this image.")
        # print(path)
        return  -2


    results = face_mesh.process(rgb_image)

    # Landmark indices for both eyes
    LEFT_EYE_LANDMARKS = [33, 133, 160, 159, 158, 157, 173, 246, 161, 144, 145, 153, 154, 155, 133, 474, 475, 476, 477]
    RIGHT_EYE_LANDMARKS = [362, 263, 387, 386, 385, 384, 398, 466, 388, 390, 373, 374, 380, 381, 382, 362, 469, 470, 471, 472]
    ALL_EYE_LANDMARKS = LEFT_EYE_LANDMARKS + RIGHT_EYE_LANDMARKS

    if results.multi_face_landmarks:
        h, w, _ = image.shape
        landmarks = results.multi_face_landmarks[0].landmark

        # Get (x, y) coordinates of all eye landmarks
        eye_points = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in ALL_EYE_LANDMARKS]

        # Compute bounding box
        x_coords, y_coords = zip(*eye_points)
        x_min_eye, x_max_eye = min(x_coords), max(x_coords)
        y_min_eye, y_max_eye = min(y_coords), max(y_coords)

        # Add padding
        padding_x = int(w*0.08) #100
        padding_y = int(h*0.23) #50
        padding_y = int((y_max_eye-y_min_eye)*2.5) #50
        padding_y_top = int((y_max_eye-y_min_eye)*2.75) #50
        padding_y_bottom = int((y_max_eye-y_min_eye)*2.5) #50
        
        # print(w, h)
        # print(f'w: {w}, h: {h}')
        # print(f'x_min_eye: {x_min_eye}, x_max_eye: {x_max_eye}, y_min_eye: {y_min_eye}, y_max_eye: {y_max_eye}')
        # print(f'x_min: {x_min}, x_max: {x_max}, y_min: {y_min}, y_max: {y_max}')
        # print()

        x_min = max(x_min_eye - padding_x, 0)
        y_min = max(y_min_eye - padding_y_bottom, 0)
        x_max = min(x_max_eye + padding_x, w)
        y_max = min(y_max_eye + padding_y_top, h)
        
        # Crop the region
        eye_crop = image[y_min:y_max, x_min:x_max]
        cv2.imwrite(f'{save_path}/{path.split('/')[-1]}', eye_crop)
    else:
        print("No face landmarks detected.")
        return  -1
    return 0

img_no_landmark = []
img_cvt_error = []

i = 0
for p in tqdm(img_paths):
    success = extract_eye_region(p, path2save)
    if success == -1: img_no_landmark.append(p)
    elif success == -2: img_cvt_error.append(p)
    #if i > 4: break
    #i+= 1
